# FSF-NEODAAS Field Spectroscopy Processing Practical - 2022

## Stated Aims

This notebook will take you through the basic steps of visualising data from a field spectrometer, and will act as the basis for different tutorial sets for you to work on later in the session.  The data in this set was collected during a survey of a beach in Scotland, 2022. 
Three different sets of spectra were acquired -- one set of the rocky shoreline, named "rocks", one of the beach itself, named "sand", and one of the seaweed vegetation strandline, named "seaweed".

## Introduction to Python

Python is a computing language that is extensively used in the scientific community. While it may not be as effective for computationally demanding tasks as some other languages, such as C or Mathematica, its open source nature, large number of freely available and easy to install modules, and clear, easy to understand syntax, make it ideal for entry level (and beyond) scientific computing. 
In particular, **the Jupyter Notebook**, developed by Project Jupyter, is an excellent tool for both learning and demonstrating Python. Jupyter Notebooks are made up of **cells**, small snippets of code that can be run consecutively, allowing you to develop -- and to understand -- Python scripts. We'll be using it to teach you more about Python and its use in the processing of field spectroscopy data.

## Using as part of the tutorial sets

This workbook is designed such that you can use the functions and methodology set out here as part of the questions asked in the tutorial sets presented later in the session.      

You can also paste your own data acquired in your work into the "data" folder and run through the processing with minimal modification. If you would like assistance in doing this in the future please email us at fsf@nerc.ac.uk


### First Steps -- Importing Modules

The first stage in any workflow in Python is to import the modules (which are collections of functions) that you will be using. In this analysis, we import a number of modules, the function of which is described in the comments (the light cyan text prefixed by the hash symbol). Note, modules must be installed first before use -- this can be handled by the **conda** package manager, or the **pip** package manager (please get in touch with us if you require help in setting this up). Here, the most important module to note is **SpecDAL**, a module designed to handle and process field spectroscopy data. To run the cell, **press Shift and ENTER**, or press **Run** in the ribbon above. 

In [ ]:
!pip install git+https://github.com/NERC-FSF/FieldSpecUtils.git -U
import os    #a module that allows us to "talk" with our operating system, neccessary for handling paths, files and so on
from pathlib import Path #a more specific path handling module that allows us to quickly create path names
from specdal import Collection, Spectrum, read #We import our main package, specdal, and ask to only import certain key functions
import pandas as pd   #Pandas is a powerful module for the handling of data sets
from matplotlib import pyplot as plt   #matplotlib emulates the functions of MATLAB. Here, we tell matplotlib to only import one of it's functions, the pyplot function, and to import it with the identifier 'plt'
from matplotlib.pyplot import ylabel, xlabel, title, legend   #We go even further by asking pyplot to only import some of it's subfunctions
from scipy.signal import savgol_filter   #We will use this to smooth our data
import numpy as np #We require numpy for integration functions that will be used during convolution
import FieldSpecUtils #NERC FSF utilities for the derivation of vegetiation indices and convultion of hyperspectral data

### SpecDAL
We will now use the SpecDAL package to read our .sig files, and assign them to a collection of spectra which we can analyse and process further. Our .sig files are located in the **Data** directory.

In [ ]:
BeachEndMembers = Collection(name='BeachEndMembers', directory ='Data')

Let's now take a look at this spectra collection. First of all, let's have a look at the header information for the
first 3 spectra in our collection. This not only prints the data itself (in the "measurements" category), but also the metadata associated with the file, such as the measurement type (here, pct_reflect refers to this being reflectance measurements), and such things as GPS information, which can be useful when matching the GPS co-ordinates with imagery.

In [ ]:
print(type(BeachEndMembers.spectra))
for s in BeachEndMembers.spectra[0:3]:
    print(s)

We can take a visual look at the data too...

In [ ]:
BeachEndMembers.plot(figsize=(15, 6), legend = True, ylim=(0,1), xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Reflectance")
plt.legend(bbox_to_anchor=(1.05, 1), loc = "upper left")
plt.show()

### Grouping Data 
It can be difficult to assess your data by viewing all spectra at once. 
We will group the data based on the vegetation type using SpecDAL's groupby function. 
This groups data based on their file names, in this case the name before the "_" separator e.g. "sand".

In [ ]:
groups = BeachEndMembers.groupby(separator='.', indices=[0])
group_names = list(groups.keys())
print(group_names)

We can now limit our graph to show only one end member type. Run this cell, then change **'seaweed'** to another one of the group names to plot that data.

In [ ]:
groups['seaweed'].plot(figsize=(15, 6), legend = True, ylim=(0,1), xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Reflectance")
plt.legend(bbox_to_anchor=(1.05, 1), loc = "upper left")
plt.show()

We can then average each of these groups to produce a collection of means, called "means". We will be returning to this collection later on, but for now, let's continue to look at individual spectra...

In [ ]:
means = Collection(name='means')
for group_key, group_collection in groups.items():
    means.append(group_collection.mean())

means.plot(title='Group means', figsize=(15, 6), ylim=(0, 1),xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Spectral Reflectance")
plt.show()

Notice that there are some odd features to the data, particularly the zig-zags at ~1000 nm, and the noisy regions at ~1800 nm. We will discuss what these issues are, and how to resolve them, in the following processing stages.

### Interpolation

Look again at the print out for your data spectra, using **data.head(10)** function to print off the first 10 rows of data. 

In [ ]:
BeachEndMembers.data.head(10)

For SVC instruments, the steps between the wavelengths correspond to the **spectral resolution** of the instrument, and are not resolved to 1 nm spacing.
We want to interpolate reflectance measurements that correspond to wavelengths with 1.0 nm spacing, a process called **interpolation**. (Note -- some spectrometers, such as the ASD FieldSpec Pro, provide their data pre-interpolated, so watch for the .asd file format in later tutorials and in the practical field session!)
This can be done using specdal as follows with the **interpolate** function (note -- ignore the error regarding monotonic functions. This is due to specdal not being updated to reflect current Python functions)  -- 

In [ ]:
BeachEndMembers.interpolate(spacing=1, method='linear')
BeachEndMembers.data.head(10)

### Overlap Stitching and Jump Correction

A typical field spectrometer covering the 350 nm to 2500 nm range consists of *three* spectrometers -- one covering the VNIR range (350 - 1000 nm), and two for the SWIR range (usually classed as SWIR-A, covering 1000 nm to 1800 nm, and SWIR-B, covering the 1800 nm to 2500 nm range. These spectrometers overlap in their ranges, leading to the zig-zags which you can see in the plots. We can correct this using specdal's **overlap stitching** function, **stitch**, and then resolve 'jumps' in the subsequent data using the **jump correction** function, **jump_correct**. Here, splices refer to the overlap regions, which in the SVC, is at 1000 nm and at 1800 nm:

In [ ]:
BeachEndMembers.stitch(method='mean')
BeachEndMembers.jump_correct(splices=[1000, 1800], reference=0)

If we now look at the data, we can see that the zig-zag features at 1000 nm are removed. Later on, we will see that the noisier features at 1800 nm are due to another cause (can you think of a reason why...?)!

In [ ]:
BeachEndMembers.plot(title='Group means', figsize=(15, 6), ylim=(0, 1),xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Spectral Reflectance")
plt.show()

###  Relative vs Absolute Reflectance
Notice from our previous graphs that the y-axes are labelled "Relative Reflectance". This is because these spectra were recorded relative to the reflectance of the white Spectralon panel. We take measurements relative to this panel to approximate the total irradiance coming from the sun and hitting the object you are interested in taking a spectral measurement of. Because the panel does not reflect 100% of the light that hits it in a completely uniform manner we need to adjust our  "Relative Reflectance" measurements using the panel's known, laboratory calibrated reflectance to convert our field measurements to absolute reflectance.

We can derive the absolute reflectance then by multiplying our field measurements by the spectral reflectance of the panel for each wavelength. We can use this file to convert our measurements to absolute reflectance, pulling data from our calibration certificate, **SRT_44.csv**. Note -- we are creating new dataframes that will host the corrected values, but the original dataframe can be corrected without having to assign a new identifier! 

In [ ]:
reference_panel = pd.read_csv("SRT_44.csv", index_col = "wavelength")
Absolute_Beach = BeachEndMembers.data.mul(reference_panel['Reflectance'], axis = 0)

Let's compare a few of the relative reflectance spectra, in the dataframe **BeachEndMembers**, with their corresponding, corrected spectra in **AbsoluteBeach**.  We can achieve this by calling specific columns in our uncorrected and corrected dataframes, concatenating them, and then comparing:

In [ ]:
Relative = BeachEndMembers.data[['rocks._0000', 'seaweed._0000',
                                  'sand._0000']].copy().add_suffix('_Relative')

Absolute = Absolute_Beach[['rocks._0000', 'seaweed._0000',
                           'sand._0000']].copy().add_suffix('_Absolute')


Relative_vs_Absolute = pd.concat([Relative, Absolute], axis=1)

Relative_vs_Absolute.head()
Relative_vs_Absolute.plot(figsize=(12, 6), legend = True, ylim=(0.0,0.45), xlim=(350, 2500))
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
xlabel("Wavelength (nm)")
ylabel("Reflectance")
plt.show()


The difference is sometimes not apparent -- this is because we are using a near "perfect" reflectance panel, quoted at 99.9% reflectance, which reflects almost all incident irradiance. The correction is therefore almost 1.0 for each value. Reflectance panels, however, do come in many different reflectance values (such as 20% reflectance), or even have values that change across the spectrum (some panels for underwater use have much lower reflectance in the SWIR range). It is always critical, then, to correct your values by the calibration certificate of the panel used -- the differences can be significant!

### Smoothing and Water Band Removal
As previously mentioned, there are some noisy regions that are not affected by the stitch and jump corrections conducted earlier. Can you see them?
Let's zoom in around two regions in the Vegetation collection -- 1350 to 1460 nm, and 1790 to 1970 nm...

In [ ]:
Absolute_Beach.plot(figsize=(8, 5), legend = None, ylim=(-1,1), xlim=(1300, 1500))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

Absolute_Beach.plot(figsize=(8, 5), legend = None, ylim=(-1,1), xlim=(1750, 2000))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

These noisy regions are caused by the absorption of solar irradiance by water vapour in the atmopshere and can interfere with classification or use in indices. If the noise isn't too bad then smoothing may be suffcient to get rid of it, otherswise we will need to remove the noisy regions. Let's smooth our data set using a Savitsky-Golay filter. The first part of the code removes the "NaN" values from our collection using the **iloc** function, which removes the region from 341 nm to 349 nm (the first 9 rows), and the 2501 nm to 2516 nm range (the rows after the 2160th row). The second part uses the Savitsky-Golay filtering technique to smooth the data.

In [ ]:
Absolute_Beach = Absolute_Beach.iloc[9:2160]
Absolute_Beach = pd.DataFrame(savgol_filter(Absolute_Beach, 41, 2, axis=0),
                             columns=Absolute_Beach.columns,
                             index=Absolute_Beach.index)

Let's take a look at how that changed things. 

In [ ]:
Absolute_Beach.plot(figsize=(8, 5), legend = None, ylim=(-1,1), xlim=(1300, 1500))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

Absolute_Beach.plot(figsize=(8, 5), legend = None, ylim=(-2,2), xlim=(1750, 2000))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

In this case, smoothing has managed to remove most of the noise from the c. 1400nm water absorption feature, but not the one at c. 1900nm. Lets completely remove the water band at c. 1900nm. We do this by assigning all the values between 1790 nm and 1970 nm with an arbitary high value (e.g. 100) in the first line of code, then replace all values of that arbitrary value in the dataframe with numpy assigned NaN:

In [ ]:
Absolute_Beach.loc[1790:1970] = 1
Absolute_Beach.replace(1,np.nan, inplace= True)

Let's plot this to see how it looks, and introduce matplotlib's save figure functionality (**tight_layout** to ensure the legend is included in the exported image, and **savefig** to save), useful for later tutorials and for producing publication quality figures (note -- you can also right click the image below and either save directly, or open in a new tab to save):

In [ ]:
Absolute_Beach.plot(figsize=(12, 6), legend = True, ylim=(0.0,0.45), xlim=(350, 2500))
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.tight_layout()
plt.savefig("Corrected Spectra.png")
plt.show()

### Putting it all together

Now that we've seen the processing functions in detail for the collection of spectra, let's now conduct the same process for our mean values, using just one cell of code:

In [ ]:
means.interpolate(spacing=1, method='linear')
means.stitch(method='mean')
means.jump_correct(splices=[1000, 1800], reference=0)
Absolute_Means = means.data.mul(reference_panel['Reflectance'], axis = 0)
Absolute_Means = Absolute_Means.iloc[9:2160]
Absolute_Means = pd.DataFrame(savgol_filter(Absolute_Means, 41, 2, axis=0),
                             columns=Absolute_Means.columns,
                             index=Absolute_Means.index)
Absolute_Means.loc[1790:1970] = 1
Absolute_Means.replace(1,np.nan, inplace= True)

Absolute_Means.plot(title='Group means', figsize=(15, 6), ylim=(0, 1),xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Absolute Spectral Reflectance")
plt.show()

We can now export this collection to a .csv, which will act as our spectral library in further processing. Unfortunately, NaN values are poorly handled by the next steps in our processing. While the region is unused, we should make sure to fill it with values. We could replace them all with 0 values, but it's neater -- and a good demonstration of interpolation -- to interpolate between missing values. 

In [ ]:
Absolute_Means.interpolate(method='linear', axis=0, inplace = True)
Absolute_Means.to_csv("Mean_plot_spectra.csv")

## Vegetation Indices
With your data now processed and converted into absolute spectral reflectance, we can look more closely at differences between vegetation types. We can use dimensionality reduction methods such as spectral indices 

https://fsf.nerc.ac.uk/resources/learning/HSI.shtml

A number of spectral indices have been designed to highlight different vegetation properties. In this next section, we will use some of them to explore differences between our vegetation types. For a full description of the indices used, please visit: https://www.l3harrisgeospatial.com/docs/NarrowbandGreenness.html

Firstly, let's import the **Vegetation_Indices** module from FSF's FieldSpecUtils module. 
**Vegetation_Indices** has a number of fucntions, which you can see by using **help(FieldSpecUtils.Vegetation_Indices)**. Each function has a description of the vegetation index it calculates. You can use the **help("module name")** feature for other modules to find out more about them.



In [ ]:
from FieldSpecUtils import Vegetation_Indices
help("FieldSpecUtils.Vegetation_Indices")

With this mind, let's calculate the Red Edge Normalized Difference Vegetation Index for the mean value of our groups in our beach survey:

In [ ]:
Vegetation_Indices.RENDVI(Absolute_Means)

### Convolution
If your research relates to specific multispectral imaging sensors e.g. Sentinel 2, it can be useful to resample 
your hyperspectral data so that it matches the bands of your specific sensor. 
We can do this by convolving the hyperspectral data to a multispectral sensor's "spectral response function". 
    
Let's convolve our hyperspectral data to match Sentinel-2 bands. To do this, we use another function in our FSF module called "Convolution". Let's import that:

In [ ]:
from FieldSpecUtils import Convolution
help("FieldSpecUtils.Convolution")

As with **Vegetation_Indices**, you can find out more about the functions included in **Convolution** by typing **help("FieldSpecUtils.Convolution")**. Notice the function **S2** and it's description. We will use **S2** to convolve our spectral library to Sentinel-2 bands. To do so, we import the spectral sesponse function for Sentinel-2, found in **Sentinel 2 SRF.csv**. It shows the relative response to specific wavelengths of light for each band of the imager.

https://fsf.nerc.ac.uk/resources/learning/SRF.shtml


In [ ]:
Bands = pd.read_csv("Sentinel 2 SRF.csv", index_col=0, header=0)

We now run the convolution function, which asks for two arguments -- our hyperspectral data, which is our beach survey means, and the bands file, which we imported above. You will be prompted whether the hyperspectral data contains areas of water band removal. Type 'Y' for yes, and press enter. This will output a .csv file in your working directory, named **Plots_with_convolved_bands**. 

In [ ]:
Convolution.S2(Absolute_Means, Bands)

Our hyperspectral data has now been convolved to multispectral bands equivalent to Sentinel-2. We can see how this looks by plotting the data output of the convolution, **Plots_with_convolved_bands.csv**:

In [ ]:
collated_convolved = pd.read_csv("Plots_with_convolved_bands.csv", index_col = 0)
collated_convolved.plot(figsize=(9, 6), legend = True, ylim=(0,1), linestyle='--', marker='o')
xlabel("Centre Wavelength (nm) of Band with Band Number")
ylabel("Absolute Reflectance")
title("Hyperspectral convolved broadband reflectance")
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
           ["490 -- Band 2", "560 -- Band 3", "665 -- Band 4", "705 -- Band 5", "740 -- Band 6",
            "783 -- Band 7", "842 -- Band 8", "865 -- Band 8a", "1610 -- Band 11", "2190 -- Band 12"],
           rotation=20)
legend(title = "Plot number and vegetation type")
plt.tight_layout()
plt.savefig("Convolved_Hyperspectral_Plot_Data_to_S2_Multispectral_Bands.pdf")
plt.show()

## Calculating Vegetation Indices using convolved data
As our hyperpsectral data convolved to Sentinel-2 equivalent multispectral bands, we can also calculate a value for the Normalized Difference Vegetation Index using band values. For Sentinel-2 bands, NDVI can be calculated, according to https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndvi/, as:

$$
\begin{align}
NDVI = \dfrac{B_8 - B_4}{B_8 + B_4}
\end{align}
$$

Where ${B_8}$ is Band 8 (centre wavelength = 842 nm) and ${B_4}$ is Band 4 (centre wavelength = 665). The cell below runs this equation using your convolved hyperspectral data, and appends the NDVI as a new row to your dataframe **collated_convolved**:

In [ ]:
collated_convolved.loc['NDVI'] = ((collated_convolved.loc['Band 8 - 842'] - collated_convolved.loc['Band 4 - 665']) / 
                                  (collated_convolved.loc['Band 8 - 842'] + collated_convolved.loc['Band 4 - 665'])) 

print(collated_convolved.loc['NDVI'])

### Conclusion

We have now gone from raw field spectroscopy data to post-processed spectra, that have then formed the basis of further investigation, such as NDVI and convolution. With these tools, you can start using your spectral data to answer many questions in different fields where spectroscopy is utilized, such as geology, ecology, or marine science! 

Use the tools and methods illustrated in this workbook to process the data contained in the set of tutorials that will be given to you after this lesson. With it, you should be able to answer the questions posed -- some of which are taken from real life situations that have gone on to scientific publication or informed public policy.

As always, you can contact the Field Spectroscopy Facility for any question regarding both acquistion and processing of spectral data, at **fsf@ed.ac.uk**. 